In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import pdb
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  2.0.0+cu118
Torchvision Version:  0.15.1+cu118


In [ ]:
df_random=pd.read_csv('/content/scene318/label318/tt0047396.txt',sep =" ")

In [ ]:
!tar -xf /content/gdrive/MyDrive/240P/train/tt0047396.tar

In [ ]:
from collections import Counter
Counter(df_random["0"])

Counter({0: 669, 1: 116, -1: 6})

In [ ]:
data_dir='/content/tt0047396'

In [ ]:
img_name = sorted(os.listdir(data_dir))

In [ ]:
!unzip /content/drive/MyDrive/scene318-20230427T183512Z-001.zip

In [ ]:
import json
import os.path as osp

In [ ]:
scene_json=pd.read_json('/content/scene318/meta/scene_movie318.json')

In [ ]:
data_root='/content/scene318/'

In [134]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd
from skimage import io, transform
transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
class MovieNetDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        self.df = self.df.loc[self.df.index.repeat(3)].reset_index(drop=True)
        self.root_dir = root_dir
        # print(len(sorted(os.listdir(self.root_dir))))
        # print(len(self.df))
        self.transform = transform
    '''def transform(self):
      data_transforms = {
     transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    }'''
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_st=[]
        for ids in range(0,6,1):
          if ids+idx<0 or ids+idx>=2370:
            continue
          img_name = sorted(os.listdir(self.root_dir))[idx+ids]
          
          image = io.imread(self.root_dir+"/"+img_name)
          #print(type(image))
          image = self.transform(image)
          image_st.append(image)
        #print(image)
        image=torch.stack(image_st)
        labels = np.array([self.df['label'][idx+3]])

        sample = {'image': torch.tensor(image), 'label': torch.tensor(labels)}

        
        return sample

In [135]:
face_dataset = MovieNetDataset(csv_file='/content/gdrive/MyDrive/tt0047396.csv',
                                    root_dir='/content/tt0047396',transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

fig = plt.figure()

# for i in range(len(face_dataset)):
#     sample = face_dataset[i]
#     print(sample['image'].shape)
#     print(i, sample['image'].shape, sample['label'])


<Figure size 640x480 with 0 Axes>

In [221]:
class Cos(nn.Module):
    def __init__(self):
        super(Cos, self).__init__()
        self.shot_num = 6
        self.channel = 512
        self.conv1 = nn.Conv2d(1, self.channel, kernel_size=(self.shot_num//2, 1))

    def forward(self, x):  # [batch_size, seq_len, shot_num, feat_dim]
        #print(x.shape)
        x = x.view(-1, 1, x.shape[2], x.shape[3])
        #print("cos_reshape")
        #print(x.shape)
        part1, part2 = torch.split(x, [self.shot_num//2]*2, dim=2)
        #print(part1.shape)
        # batch_size*seq_len, 1, [self.shot_num//2], feat_dim
        part1 = self.conv1(part1).squeeze()
        part2 = self.conv1(part2).squeeze()
        #print(part1.shape)
        x = F.cosine_similarity(part1, part2, dim=1)  # batch_size,channel
        # x = F.cosine_similarity(part1, part2, dim=1)  # batch_size,channel
        # x = F.cosine_similarity(part1, part2, dim=2)  # batch_size,channel
        # x = F.cosine_similarity(part1, part2, dim=3)  # batch_size,channel

        return x

In [222]:
class BNet(nn.Module):
    def __init__(self):
        super(BNet, self).__init__()
        self.shot_num = 6
        self.channel = 512
        self.conv1 = nn.Conv2d(1, self.channel, kernel_size=(6, 1))
        self.max3d = nn.MaxPool3d(kernel_size=(self.channel, 1, 1))
        self.cos = Cos()
        model_ft = models.resnet18(pretrained=True)
        model_ft.fc = nn.Identity()
        model_ft.eval()
        self.feat_extractor = model_ft

    def forward(self, x):  # [batch_size, seq_len, shot_num, 3, 224, 224]
        #print(x.shape)
        #print(type(x))
        feat = self.feat_extractor(x.float())
        #print(feat.shape)
        #feat = feat.view(feat.size(0), -1)
        feat =  feat.view(1, 1, 6, -1)
        
        #print("feat",feat)
        # [batch_size, seq_len, shot_num, feat_dim] 
        context=feat
        context = feat.view(feat.shape[0]*feat.shape[1], 1, feat.shape[-2], feat.shape[-1])
        #print("context",context)
        context = self.conv1(context)
        # batch_size*seq_len,sim_channel,1,feat_dim
        context = self.max3d(context)
        # batch_size*seq_len,1,1,feat_dim
        context = context.squeeze()
        sim = self.cos(feat)
        #print(context.shape,sim.shape)
        bound = torch.cat((context, sim))
        return bound


In [223]:
class LGSSone(nn.Module):
    def __init__(self, mode="image"):
        super(LGSSone, self).__init__()
        self.seq_len = 1
        self.num_layers = 1
        self.lstm_hidden_size = 512
        if mode == "image":
            self.bnet = BNet()
            self.input_dim = (512+
                512)

        self.fc1 = nn.Linear(5632, 100)
        self.fc2 = nn.Linear(100, 1)

    def forward(self, x):
        #print(x)
        x = self.bnet(x)
        x = x.view(-1, self.seq_len, x.shape[-1])
        # torch.Size([128, seq_len, 3*channel])
        #self.lstm.flatten_parameters()
        #out, (_, _) = self.lstm(x, None)
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        #print(out.shape)
        #out=out.flatten()
        out = F.relu(self.fc1(x))
        out = self.fc2(out)
        #out = out.view(-1, 2)
        return torch.sigmoid(out)

In [224]:
class LGSS_image(nn.Module):
    def __init__(self):
        super(LGSS_image, self).__init__()
        #self.mode = cfg.dataset.mode
        #if 'image' in self.mode:
        self.bnet_image = LGSSone("image")

    def forward(self, img):
        out = 0
        image_bound = self.bnet_image(img)
        out = image_bound
        return out

In [226]:
model=LGSS_image()

In [227]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(2):
  c=0
  for i in range(0,2000,1):
    '''
    imgs=[]
    labels=[]
    for j in range(i,i+10):
      img=(face_dataset[j]['image'])
      imgs.append(img)
      label=(face_dataset[j]['label'])
      labels.append(label)
    imgs=torch.stack(imgs)'''
    img=(face_dataset[i]['image'])
    labels=(face_dataset[i]['label'])

    imgs=torch.reshape(imgs,(60, 3, 224, 224))
    labels=torch.tensor(labels)
    #print(img.dtype)
    #print(img,labels)
    optimizer.zero_grad()
    output=model(imgs)
    #print(output.flatten())
    #print(labels)
    output=output.flatten()
    for k in range(1):
      if torch.round(output[k])==labels[k]:
        c+=1
    loss=criterion(output,labels.float())
    loss.backward()
    optimizer.step()
    #print(c)
    #print(loss.item())
    if i%50==0 and i>0:
      print(f"Epoch: {epoch},batch_processed: {i}/2000, train_acc: {c/i}")


In [ ]:
torch.save(model,'/content/gdrive/MyDrive/model_cnn.h5')

In [59]:
model=torch.load('/content/gdrive/MyDrive/model_cnn.h5')

In [ ]:
model.eval()

In [61]:
import warnings
warnings.filterwarnings("ignore")

In [62]:
from sklearn.metrics import average_precision_score

In [ ]:
c=0
preds=[]
l=[]
for i in range(2001,2350,10):
    imgs=[]
    labels=[]
    #c=0
    for j in range(i,i+10):
      img=(face_dataset[j]['image'])
      imgs.append(img)
      label=(face_dataset[j]['label'])
      labels.append(label)
    imgs=torch.stack(imgs)
    imgs=torch.reshape(imgs,(60, 3, 224, 224))
    labels=torch.tensor(labels)
    #print(img.dtype)
    #print(img,labels)
    optimizer.zero_grad()
    output=model(imgs)
    #print(output.flatten())
    #print(labels)
    output=output.flatten()
    threshold=0.006
    #print(output)
    output = torch.where(output < threshold, torch.zeros_like(output), torch.ones_like(output))
    #print(output)
    #print(labels)
    for k in range(10):
      #print(output[k],labels[k])
      if output[k]==labels[k]: 
        c+=1
    gt = torch.round(output)
    #print(gt.detach().numpy(),labels)
    #break
    preds.append(gt.detach().numpy())
    l.append(labels.numpy())
    precision = average_precision_score(gt.detach().numpy(),labels) 
    #loss=criterion(output,labels.float())
    #loss.backward()
    #optimizer.step()
    #print(c,i)
    #print(loss.item())
    #print(f"Epoch: {epoch},batch_processed: {i-2000/350}, test_acc: {c/(i-2000)},precision: {precision}")
    if i%10==1 and i>2001:
      print(f"batch_processed: {(i-2000)/350}, test_acc: {c/(i-1991)},precision: {precision}")

batch_processed: 0.03142857142857143, test_acc: 0.8,precision: 0.2
batch_processed: 0.06, test_acc: 0.8,precision: 0.2
batch_processed: 0.08857142857142856, test_acc: 0.8,precision: 0.2
batch_processed: 0.11714285714285715, test_acc: 0.8,precision: 0.2
batch_processed: 0.1457142857142857, test_acc: 0.8,precision: 0.2
batch_processed: 0.1742857142857143, test_acc: 0.8,precision: 0.2
batch_processed: 0.20285714285714285, test_acc: 0.8,precision: 0.2
batch_processed: 0.23142857142857143, test_acc: 0.8,precision: 0.2
batch_processed: 0.26, test_acc: 0.8,precision: 0.2
batch_processed: 0.2885714285714286, test_acc: 0.7909090909090909,precision: 0.3
batch_processed: 0.3171428571428571, test_acc: 0.7916666666666666,precision: 0.5444444444444444
batch_processed: 0.3457142857142857, test_acc: 0.7846153846153846,precision: 0.43333333333333335
batch_processed: 0.3742857142857143, test_acc: 0.7357142857142858,precision: 0.24166666666666667
batch_processed: 0.40285714285714286, test_acc: 0.72,preci

In [ ]:
AP = average_precision_score(preds,l)

In [ ]:
AP

0.3274055829228243